In [3]:
import crocoddyl
import pinocchio
import numpy as np
import example_robot_data as robex
from tp3.meshcat_viewer_wrapper import MeshcatVisualizer
import tp7.croco_utils as crocutils

Load and set the robot 

In [4]:
robot = robex.load('talos_arm')
robot_model = robot.model

robot_model.armature =np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.])*5
robot_model.q0 = np.array([3.5,2,2,0,0,0,0])
robot_model.x0 = np.concatenate([robot_model.q0, np.zeros(robot_model.nv)])
robot_model.gravity *= 0

viz = MeshcatVisualizer(robot)
viz.display(robot_model.q0)
viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


Targets definition

In [5]:
FRAME_TIP = robot_model.getFrameId("gripper_left_fingertip_3_link")
goals = np.array([[.4,0,.4],[.4,.4,.4],[0.,.0,.4],[0,.4,.4]])
viz.addBox('world/goal',[.1,.1,.1],[0,1,0,1])
viz.applyConfiguration('world/goal',[0.4,0,.4,0,0,0,1])
viz.addBox('world/goal2',[.1,.1,.1],[0,1,0,1])
viz.applyConfiguration('world/goal2',[0.4,0.4,.4,0,0,0,1])
viz.addBox('world/goal3',[.1,.1,.1],[0,1,0,1])
viz.applyConfiguration('world/goal3',[0,0,.4,0,0,0,1])
viz.addBox('world/goal4',[.1,.1,.1],[0,1,0,1])
viz.applyConfiguration('world/goal4',[0,0.4,.4,0,0,0,1])
viz.display(robot_model.q0)

Models creation

In [6]:
state = crocoddyl.StateMultibody(robot_model)
runningCostModels = [crocoddyl.CostModelSum(state) for i in range(4)]
terminalCostModels = [crocoddyl.CostModelSum(state) for i in range(4)]

Costs definition

In [7]:
### Cost for reaching the target
Mref = crocoddyl.FramePlacement(FRAME_TIP,pinocchio.SE3(np.eye(3), goals[0]))
prefs = [crocoddyl.FrameTranslation(FRAME_TIP,goals[i]) for i in range(4)]
goalTrackingCosts = [crocoddyl.CostModelFrameTranslation(state, prefs[i]) for i in range(4)]

### Cost for regularizing the state about robot_model.x0
weights=crocoddyl.ActivationModelWeightedQuad(np.array([1,1,1,1,1,1,1, 1,1,1,1,2,2,2.]))
xRegCost = crocoddyl.CostModelState(state,weights,robot_model.x0)

### Cost for keeping the control low
uRegCost = crocoddyl.CostModelControl(state)

for i in range(4):
    runningCostModels[i].addCost("gripperPose", goalTrackingCosts[i], 0.001)
    runningCostModels[i].addCost("xReg", xRegCost, 5e-2)
    runningCostModels[i].addCost("uReg", uRegCost, 1e-5)

    terminalCostModels[i].addCost("gripperPose", goalTrackingCosts[i], 40)

actuationModel = crocoddyl.ActuationModelFull(state)
dt = 1e-2
runningModels = [crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, runningCostModels[i]), dt) for i in range(4)]

for i in range(4):
    runningModels[i].differential.armature = robot_model.armature

terminalModels = [crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, terminalCostModels[i]), 0.) for i in range(4)]

for i in range(4):
    terminalModels[i].differential.armature = robot_model.armature

Shooting problem creation

In [8]:
T = 100

seq0 = [runningModels[0]]*T + [terminalModels[0]]
seq1 = [runningModels[1]]*T + [terminalModels[1]]
seq2 = [runningModels[2]]*T + [terminalModels[2]]
seq3 = [runningModels[3]]*T 
problem = crocoddyl.ShootingProblem(robot_model.x0,seq0+seq1+seq2+seq3,terminalModels[3])

DDP solver creation

In [9]:
ddp = crocoddyl.SolverDDP(problem)
ddp.setCallbacks([
    crocoddyl.CallbackLogger(),
    crocoddyl.CallbackVerbose(),
])
ddp.solve([],[],1000)

True

Visualization of the solution

In [10]:
crocutils.displayTrajectory(viz,ddp.xs,ddp.problem.runningModels[0].dt,12)